<a href="https://colab.research.google.com/github/reparadise/MachinelearningProjectAifel/blob/main/03_Vectorization_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트의 분포로 벡터화 하기

텍스트 분포를 이용한 텍스트의 벡터화 방법들(BoW, DTM, TF-IDF, LSA, LDA)을 실습을 통해 익혀보고, 텍스트 분포 기반으로 구현된 토큰화 기법에 대해서도 살펴본다.


- 들어가며
- 단어빈도를 이용한 벡터화
  - BOW 의미과 구현
  - DTM & 코사인 의미와 구현, 한계점
  - TF-IDF 의미와 구현
- LSA & LDA
  - LSA 의미와 구현
  - LDA 의미와 구현
- 비지도 학습 토크나이저
  - 형태소 분석기의 문제점
  - soynlp


# 단어 빈도를 이용한 벡터화

## Bag of Words(BoW)

![https://koushik1102.medium.com/nlp-bag-of-words-and-tf-idf-explained-fd1f49dce7c4](https://miro.medium.com/max/1400/1*ZBBlFr8_uj5owi4Z_YhzOw.jpeg)

어순을 반영하지 못함

keras Tokenizer로 구현

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


scikit-learn CountVectorizer 활용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


In [ ]:
print('단어장(Vocabulary)의 크기 :', len(vector.vocabulary_))

단어장(Vocabulary)의 크기 : 10


## DTM과 코사인 유사도

DTM(Document-Term Matrix)

여러 문서의 BoW를 하나의 행렬로 구현

![https://medium.com/natural-language-processing-machine-learning/nlp-for-beginners-how-simple-machine-learning-model-compete-with-the-complex-neural-network-on-b9f7f93c79e6](https://miro.medium.com/max/1400/1*8Ru42sJN1AyRHqqsxoKW_g.png)

각 행은 많은 값이 0으로 구성

각 문서 간 유사도를 구할 수 있게 됨

![https://wikidocs.net/24603](https://wikidocs.net/images/page/24603/%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84.PNG)

In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

docs = ['I like dog', 'I like cat' , 'I like cat I like cat']

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [ ]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 각 단어의 빈도를 bow에 저장

print("DTM :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

DTM : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


- 장점
  - BoW를 비교할 수 있음
- 단점
  - 차원의 저주(0이 많음 == 희소행렬)
  - 빈도에 치중(the가 많은 문장은 비슷한 문장?)
    - 단어간 중요도를 확인해서 접근하는 방법이 필요!


## TF-IDF

Term Frequency-Inverse Document Frequency

단어 빈도 - 역 문서 빈도

- 모든 문서에 자주 등장하면 중요도가 낮음
- 특정 문서에 자주 등장하면 중요도가 높음

불용어 노이즈를 줄여주는 것을 목표로 함

TF == Term Frequncey == 

TF-IDF 식

![https://towardsdatascience.com/introduction-to-natural-language-processing-for-text-df845750fb63](https://miro.medium.com/max/1400/1*V9ac4hLVyms79jl65Ym_Bw.png)

[TF-IDF 설명](https://www.youtube.com/watch?v=Rd3OnBPDRbM)


In [ ]:
from math import log
import pandas as pd

In [ ]:
docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

In [ ]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [ ]:
N = len(docs) # 총 문서의 수
N

3

In [ ]:
def tf(t, d):
    return d.count(t)
 
def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1)) + 1
 
def tfidf(t, d):
    return tf(t,d)* idf(t)

In [ ]:
result = []
for i in range(N): # 각 문서에 대해서 아래 명령을 수행
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    
    result[-1].append(tf(t, d))
      
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [ ]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [ ]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


scikit-learn TFidVectorizer 사용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


결과값이 다른 이유

- [tf-idf 식이 변경됨](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html)

- 장점
  - 구현하기 쉬움
  - 단어 중요도를 잘 나타냄
- 단점
  - 단어의 유사도를 보지 않음
  - 뜻을 확인하지 못함
    - 한식 != 김치는 한식 != 김치


# LSA와 LDA 
LSA는 DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶는 반면에, LDA는 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합 확률로 추정하여 토픽을 추출합니다.

## LSA

- 단어의 의미를 표현하기 위해서는?
- LSA(Latent Semantic Analysis)!

잠재 의미 분석

단어와 단어, 문서와 문서, 단어와 문서 사이 유사성 점수 판단

특이값 분해(SVD : Singular Value Decompotion)를 사용함

[공돌이 수학정리 노트 영상](https://www.youtube.com/watch?v=cq5qlYtnLoY)


![https://www.researchgate.net/figure/The-Singular-Value-Decomposition-SVD-of-an-m-n-rectangular-matrix-A_fig1_287246825](https://www.researchgate.net/profile/Naji-Albatayneh/publication/287246825/figure/fig1/AS:410012099858432@1474765910459/The-Singular-Value-Decomposition-SVD-of-an-m-n-rectangular-matrix-A.png)

k가 클수록 범용적임

k가 적을수록 노이즈가 잡힘

- [특이값 분해](https://datascienceschool.net/02%20mathematics/03.04%20%ED%8A%B9%EC%9E%87%EA%B0%92%20%EB%B6%84%ED%95%B4.html)
- [특히값 분해 영상](https://www.youtube.com/watch?v=vxJ1MzfvL5w)

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
print('=3')

=3


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
csv_filename = 'abcnews-date-text.csv'
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('abcnews-date-text.csv', <http.client.HTTPMessage at 0x7f67807eaa90>)

In [ ]:
data = pd.read_csv(csv_filename, error_bad_lines=False)
data.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


(1082168, 2)

In [ ]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [ ]:
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [ ]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

In [ ]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [ ]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [ ]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc

In [ ]:
print(train_data[:5])

['aba decide community broadcast licence', 'act fire witness must aware defamation', 'call infrastructure protection summit', 'air staff aust strike pay rise', 'air strike affect australian travellers']


In [ ]:
# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)
print('=3')

=3


In [ ]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


scikit-learn TruncatedSVD

In [ ]:
from sklearn.decomposition import TruncatedSVD

lsa_model = TruncatedSVD(n_components = 10)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20666003e-02, -3.68758849e-03,  1.81963486e-02, ...,
         2.83832809e-03,  1.59013471e-05,  1.54105845e-02],
       [ 2.90488818e-02, -1.08788922e-02,  1.81371795e-02, ...,
         5.67337274e-04, -7.88641942e-03, -1.00086207e-02],
       [ 5.04511925e-03, -2.01134333e-03,  9.72678271e-03, ...,
        -2.51848132e-03,  1.28824752e-03,  3.31906929e-03],
       ...,
       [ 2.98466465e-02,  4.18342092e-03,  2.46923789e-02, ...,
         2.96607247e-02,  1.15966655e-02,  1.15764561e-02],
       [ 6.12804560e-02, -4.94550224e-03,  1.38538326e-01, ...,
         8.45456143e-01,  9.13292476e-01, -1.70470415e-01],
       [ 7.14186130e-02,  2.89854861e-02,  1.53167813e-03, ...,
        -1.01203203e-02, -3.11591625e-02, -1.09439506e-02]])

In [ ]:
print(lsa_model.components_.shape)

(10, 5000)


In [ ]:
terms = c_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74631), ('man', 0.45341), ('charge', 0.21111), ('new', 0.14078), ('court', 0.1115)]
Topic 2: [('man', 0.69394), ('charge', 0.30052), ('court', 0.16793), ('face', 0.1151), ('murder', 0.10605)]
Topic 3: [('new', 0.83708), ('plan', 0.23627), ('say', 0.18268), ('govt', 0.11088), ('council', 0.11047)]
Topic 4: [('say', 0.74143), ('plan', 0.3577), ('govt', 0.16847), ('council', 0.13242), ('urge', 0.07805)]
Topic 5: [('plan', 0.73263), ('council', 0.1825), ('govt', 0.14521), ('urge', 0.0937), ('water', 0.06916)]
Topic 6: [('govt', 0.57054), ('court', 0.26016), ('urge', 0.25655), ('fund', 0.19462), ('win', 0.15613)]
Topic 7: [('charge', 0.51203), ('court', 0.48357), ('face', 0.33402), ('murder', 0.12883), ('plan', 0.11185)]
Topic 8: [('win', 0.58086), ('court', 0.40289), ('kill', 0.20076), ('crash', 0.12344), ('cup', 0.0828)]
Topic 9: [('win', 0.65402), ('charge', 0.4447), ('australia', 0.08285), ('cup', 0.07754), ('world', 0.06871)]
Topic 10: [('council', 0.86596), ('fu

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## LDA

잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

문장에서 주제를 찾아내고 주제에서 단어를 찾아냄

이후 단어가 어떤 주제인지 유추함

[LDA Simulation](https://lettier.com/projects/lda-topic-modeling/)

![https://heeya-stupidbutstudying.tistory.com/entry/ML-%EC%9E%A0%EC%9E%AC-%EB%94%94%EB%A6%AC%ED%81%B4%EB%A0%88-%ED%95%A0%EB%8B%B9-%EA%B0%9C%EC%9A%94-LDALatent-Dirichlet-Allocation-1](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbLpviW%2FbtrwPbWCCfI%2FylzwBUSuSEPkIpXh9OR080%2Fimg.png)

[글로 설명하는 LDA](https://wikidocs.net/30708)

- 바나나를 먹어요
- 귀여운 강아지가 좋아요
- 귀여운 강아지가 바나나를 먹어요

- 100% 100% 60:40%

[동영상 강의 LDA](https://www.youtube.com/watch?v=4AGjlcEQ6I8)
- 어렵..

LSA는 DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶는 반면에, LDA는 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합 확률로 추정하여 토픽을 추출합니다.

In [ ]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

In [ ]:
# LDA의 결과 토픽과 각 단어의 비중을 출력합시다
terms = tfidf_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',

# 비지도 학습 토크나이저

## 형태소 분석기와 단어 미등록 문제

In [ ]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [ ]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


사과를 잘 잘라내지 못함

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.8 MB/s 
     |████████████████████████████████| 453 kB 46.0 MB/s 


In [ ]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [ ]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


모두의연구소는 한 단어지만 인식하지 못함

## Soynlp

위 문제를 해결하기 위해 다음 방법을 사용함

응집 확률(cohesion probability) & 브랜칭 엔트로피(branching entropy)

In [ ]:
import urllib.request

txt_filename = '2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('2016-10-20.txt', <http.client.HTTPMessage at 0x7f79462b6b10>)

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 4.0 MB/s 


In [ ]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [ ]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [ ]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 3.289 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### 응집 확률(cohesion probability)

$$cohesion(n) = \big( \prod_{i=1}^{n-1} P(c_{1:i+1}|c_{1:i})\big)^{ \frac{1}{n-1}}$$



![https://wikidocs.net/92961](https://wikidocs.net/images/page/92961/%EC%88%98%EC%8B%9D2.png)

위와 같음

In [ ]:
word = '반포한강공원에'
for i in range(3, len(word) + 1):
  print(word[:i])
  print(word_score_table[word[:i]].cohesion_forward)

반포한
0.08838002913645132
반포한강
0.19841268168224552
반포한강공
0.2972877884078849
반포한강공원
0.37891487632839754
반포한강공원에
0.33492963377557666


### 브랜칭 엔트로피(branching entropy)

In [ ]:
word = '반포한강공원에'
for i in range(3, len(word) + 1):
  print(word[:i])
  print(word_score_table[word[:i]].right_branching_entropy)

반포한
-0.0
반포한강
-0.0
반포한강공
-0.0
반포한강공원
1.3542944153448395
반포한강공원에
0.9922819748525737


- 반포 다음은 무조건 한
- 반포한 다음은 무조건 한
- 반포한강 다음은 무조건 공
- 반포한강공 뒤에는 다른 가능성이 있음

띄어쓰기로 잘 나뉜 문장은 L토크나이저(LTokenizer) 사용

즉, left가 높은 방향으로 나눔

In [ ]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

최대 점수 토크나이저(MaxScoreTokenizer)

띄어쓰기가 없는 경우 좋은 성과를 냄

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']